In [151]:
import pandas as pd
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
INPUT_DATA_FOLDER = "isw_data"
DATA_FILE = "table_of_data.csv"

OUTPUT_FOLDER = "isw_data"
OUTPUT_DATA_FILE = "table_of_data_preprocessed.csv"


C:\Users\Богдан\AppData\Local\Temp\ipykernel_3012\2446677717.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [152]:
#df = pd.read_csv(f"{INPUT_DATA_FOLDER}/{DATA FILE)", sep=";")
df = pd.read_csv(f"{DATA_FILE}")

In [153]:
df.head(10)

,Unnamed: 0,date,title,text_title,text_main
0,0,2022-02-25,Russia-Ukraine Warning Update: Russian Offensi...,"Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t..."
1,1,2022-02-26,Russia-Ukraine Warning Update: Russian Offensi...,"Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t..."
2,2,2022-02-27,Russia-Ukraine Warning Update: Russian Offensi...,"Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t..."
3,3,2022-02-28,"Russian Offensive Campaign Assessment, Februar...","Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t..."
4,4,2022-03-01,"Russian Offensive Campaign Assessment, March 1...","Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t..."
5,5,2022-03-02,"Russian Offensive Campaign Assessment, March 2...","Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t..."
6,6,2022-03-03,Ukraine Conflict Update 14 | Institute for the...,"Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t..."
7,7,2022-03-04,Ukraine Conflict Update 15 | Institute for the...,"Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t..."
8,8,2022-03-05,"Explainer on Russian Conscription, Reserve, an...","Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t..."
9,9,2022-03-06,Ukraine Conflict Update 16 | Institute for the...,"Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t..."


In [154]:
test_row = df.iloc[0]
page_html_text = test_row["text_main"]
print(page_html_text)

<div class="field field-name-body field-type-text-with-summary field-label-hidden"><div class="field-items"><div class="field-item even" property="content:encoded"> <p align="center" style="text-align: left;"><strong>Mason Clark, George Barros, and Kateryna Stepanenko</strong></p>
<p align="center" style="text-align: left;"><strong>February 25, 3:00 pm EST</strong></p>
<p style="text-align: left;"><strong>Russian forces entered major Ukrainian cities—including Kyiv and Kherson—for the first time on February 25. </strong>Russian forces’ main axes of advance focused on Kyiv (successfully isolating the city on both banks of the Dnipro River). Russian military operations along Ukraine’s northern border have been less well-planned, organized, and conducted than those emanating from Crimea. They have also been less successful so far. The divergence in performance likely arises in part from differences in the composition and organization of the Russian ground forces elements in the Western Mi

In [155]:
#display(HTML(page_html_text))

In [156]:
def remove_names_and_date(page_html_text):
    parsed_html = BeautifulSoup(page_html_text)
    p_lines = parsed_html.findAll('p')
    
    min_sentense_word_count = 13
    p_index = 0
    
    #find first long sentense
    for p_line in p_lines:
        
        strong_lines = p_line.findAll('strong')
        if not strong_lines:
            continue

        for s in strong_lines:
            if len(s.text.split(" ")) >= min_sentense_word_count:
                break
        else:
            p_index += 1
            continue
        break
    for i in range(0, p_index):
        page_html_text = page_html_text.replace(str(p_lines[i]), "")
        
    return page_html_text   

In [157]:
df['main_html_v2'] = df['text_main'].apply(lambda x: remove_names_and_date(x))

In [158]:

#display(HTML(df.iloc[0]["main_html_v2"]))

In [159]:
import re
pattern = "\[(\d+)\]"
df['main_html_v3'] = df['main_html_v2'].apply(lambda x: re.sub(pattern, "", x))
display(HTML(df.iloc[0]["main_html_v3"]))

In [160]:
df['main_html_v4'] = df['main_html_v3'].apply(lambda x: BeautifulSoup(x).text)
#display(HTML(df.iloc[0]["main_html_v4"]))

In [161]:
df['main_html_v5'] = df['main_html_v4'].apply(lambda x: re.sub(r'http(\S+.*\s)', "", x))
df['main_html_v6'] = df['main_html_v5'].apply(lambda x: re.sub(r'(©2022|©2023|2022|2023)', "", x))
df['main_html_v7'] = df['main_html_v6'].apply(lambda x: re.sub(r'\n.{5,15}\d:\d.{0,9}\n', "", x))
df['main_html_v8'] = df['main_html_v7'].apply(lambda x: re.sub('Appendix A – Satellite Imagery(.|\n)+\.', "", x)).apply(lambda x: re.sub('Click here to expand the map below.', "", x))

In [162]:
print(df.iloc[3]["main_html_v8"])

The Russian military is reorganizing its military efforts in an attempt to remedy poor planning and execution based on erroneous assumptions about Ukrainians’ will and ability to resist.  Russian operations around Kyiv remain limited as logistics and reinforcements arrive but will likely resume in greater strength in the next 24 hours. Ukrainian military leaders say that they have used the pause to strengthen Kyiv’s defenses and prepare to defend their capital in depth. The Ukrainian military likely cannot prevent Russian forces from enveloping or encircling Kyiv if the Russians send enough combat power to do so, but likely can make Russian efforts to gain control of the city itself extremely costly and possibly unsuccessful.
The Russian military has begun using area-attack weapons in the city of Kharkiv, dramatically increasing the damage to civilian infrastructure and the number of civilian casualties it is causing. It is using tube- and rocket artillery against Kharkiv, and unconfir

In [163]:
df=df.drop(['Unnamed: 0','main_html_v2','main_html_v3','main_html_v4','main_html_v5','main_html_v6','main_html_v7'],axis=1)
df.head(10)

,date,title,text_title,text_main,main_html_v8
0,2022-02-25,Russia-Ukraine Warning Update: Russian Offensi...,"Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t...",\nRussian forces entered major Ukrainian citie...
1,2022-02-26,Russia-Ukraine Warning Update: Russian Offensi...,"Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t...",\nRussian forces’ main axes of advance in the ...
2,2022-02-27,Russia-Ukraine Warning Update: Russian Offensi...,"Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t...",\nThe Russian military has likely recognized t...
3,2022-02-28,"Russian Offensive Campaign Assessment, Februar...","Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t...",The Russian military is reorganizing its milit...
4,2022-03-01,"Russian Offensive Campaign Assessment, March 1...","Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t...",Russian forces are completing the reinforcemen...
5,2022-03-02,"Russian Offensive Campaign Assessment, March 2...","Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t...",Russian forces resumed offensive operations in...
6,2022-03-03,Ukraine Conflict Update 14 | Institute for the...,"Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t...","\nwith the Critical Threats Project, AEI\nMarc..."
7,2022-03-04,Ukraine Conflict Update 15 | Institute for the...,"Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t...","\nwith the Critical Threats Project, AEI\nMarc..."
8,2022-03-05,"Explainer on Russian Conscription, Reserve, an...","Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t...",\nThe early announcement of the spring draft ...
9,2022-03-06,Ukraine Conflict Update 16 | Institute for the...,"Russian Offensive Campaign Assessment, January...","<div class=""field field-name-body field-type-t...","\nwith the Critical Threats Project, AEI\nMarc..."


In [166]:
from pathlib import Path
with open("D:/Proga/table_of_data_preprocessed.csv", "x") as file:
    filepath=Path("D:/Proga/table_of_data_preprocessed.csv")
    df=df.to_csv(filepath)

FileExistsError: [Errno 17] File exists: 'D:/Proga/table_of_data_preprocessed1.csv'